In [1]:
import pandas as pd
import numpy as np


dataset = pd.read_csv("clinical_events.csv",index_col=0)
dataset.head()

/Users/lno7761/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,clinical_event_id,clinical_event_member_id,clinical_event_name,clinical_event_time,clinical_event_member_age,member_male
1,1.0,1,ANNUAL VISIT,41,81,1
2,2.0,1,STROKE HOSPITALIZATION,641,81,1
3,3.0,1,HYPERTENSION MANAGEMENT VISIT,116,81,1
4,4.0,1,HYPERTENSION MANAGEMENT VISIT,293,81,1
5,5.0,2,ANNUAL VISIT,14,76,1


## get snapshot from 30 days to  600 days or stroke hospitalization visit day

In [2]:
df_maxtime = pd.DataFrame(dataset.groupby(['clinical_event_member_id','clinical_event_member_age','member_male'])['clinical_event_time'].max()).reset_index()

def get_member_interval_outcomes( data, hours_of_interest=30):
    df = data.drop_duplicates()
    psst = df.set_index('clinical_event_member_id')
    psst['tdindex'] = psst.apply(lambda x:np.arange(0,max(730-150,x.clinical_event_time),hours_of_interest,dtype=np.int),axis=1)
    psst=psst.explode('tdindex')
    psst['tdindex_feat']=psst['tdindex'].astype(np.int)
    psst=psst.reset_index().set_index(['clinical_event_member_id','tdindex'])
    psst["stroke_visit_180days"]=(psst['stroke_event_time'] > 0)&((psst['tdindex_feat'] + (180-1) >=psst['stroke_event_time']) & (psst['tdindex_feat'] < psst['stroke_event_time']))
    #psst = psst[((psst.clinical_event_time < 0) | ((psst.clinical_event_time>0) & (psst.clinical_event_time>psst.tdindex_feat)))]
    return psst

stroke_pats = dataset[dataset.clinical_event_name=="STROKE HOSPITALIZATION"][['clinical_event_member_id','clinical_event_time']]
stroke_pats = stroke_pats.rename(columns={'clinical_event_time':'stroke_event_time'})
tmp = df_maxtime.merge(stroke_pats,how="left").fillna(-200)
psst = get_member_interval_outcomes(tmp)


<ipython-input-2-c5f17841c4ed>:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  psst['tdindex'] = psst.apply(lambda x:np.arange(0,max(730-150,x.clinical_event_time),hours_of_interest,dtype=np.int),axis=1)
<ipython-input-2-c5f17841c4ed>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use,

In [9]:
psst

clinical_event_member_age  member_male  \
clinical_event_member_id tdindex                                           
1                        0                               81            1   
                         30                              81            1   
                         60                              81            1   
                         90                              81            1   
                         120                             81            1   
...                                                     ...          ...   
200000                   450                             63            1   
                         480                             63            1   
                         510                             63            1   
                         540                             63            1   
                         570                             63            1   

                                  clinical_event_time  stroke_event_time  \
clinical_event_member_id tdindex                                           
1                        0                        641              641.0   
                         30                       641              641.0   
                         60                       641              641.0   
                         90                       641              641.0   
                         120                      641              641.0   
...                                               ...                ...   
200000                   450                      127             -200.0   
                         480                      127             -200.0   
                         510                      127             -200.0   
                         540                      127             -200.0   
                         570                      127             -200.0   

                                  tdindex_feat  stroke_visit_180days  
clinical_event_member_id tdindex                                      
1                        0                   0                 False  
                         30                 30                 False  
                         60                 60                 False  
                         90                 90                 False  
                         120               120                 False  
...                                        ...                   ...  
200000                   450               450                 False  
                         480               480                 False  
                         510               510                 False  
                         540               540                 False  
                         570               570                 False  

[4033675 rows x 6 columns]

## get time lapsed since last visit

In [3]:
def multiIndex_pivot(df, index = None, columns = None, values = None):
    # https://github.com/pandas-dev/pandas/issues/23955
    output_df = df.copy(deep = True)
    if index is None:
        names = list(output_df.index.names)
        output_df = output_df.reset_index()
    else:
        names = index
    #print(output_df.info(verbose=True))
    output_df = output_df.assign(tuples_index = [tuple(i) for i in output_df[names].values])    
    #print(output_df.loc[output_df.duplicated(subset=['tuples_index'],keep=False),'tuples_index'].head(100))
    if isinstance(columns, list):
        output_df = output_df.assign(tuples_columns = [tuple(i) for i in output_df[columns].values])  # hashable
        output_df = output_df.pivot(index = 'tuples_index', columns = 'tuples_columns', values = values) 
        output_df.columns = pd.MultiIndex.from_tuples(output_df.columns, names = columns)  # reduced
    else:
        output_df = output_df.pivot(index = 'tuples_index', columns = columns, values = values)    
    output_df.index = pd.MultiIndex.from_tuples(output_df.index, names = names)
    return output_df


events = dataset[['clinical_event_member_id','clinical_event_name','clinical_event_time','clinical_event_id']]

cat_frames=[]
cat_frames.append(events.drop_duplicates(['clinical_event_member_id','clinical_event_time','clinical_event_name']))
cat_frames[-1]=cat_frames[-1].pipe(multiIndex_pivot, index = ['clinical_event_member_id','clinical_event_time'], columns = ['clinical_event_name'], values = 'clinical_event_time')

# create incidence (multi-hot) radiology feature
df_times=pd.concat(cat_frames,axis=1,ignore_index=True)
df_times.columns=[x for y in cat_frames for x in y.columns.get_level_values(0)]

In [4]:
df_hr=pd.concat(cat_frames,axis=1,ignore_index=True)


df_hr = df_hr.reset_index()
df_hr.clinical_event_time=np.ceil(df_hr.clinical_event_time/30.0)*30

d = {x:"last" for x in df_hr.columns}
df_hr = df_hr.groupby(['clinical_event_member_id','clinical_event_time'], as_index=True).aggregate(d).reindex(columns=df_hr.columns)
del df_hr['clinical_event_member_id'],df_hr['clinical_event_time']
df_hr

df_hr.columns=[x+'_lv' for y in cat_frames for x in y.columns.get_level_values(0)]


df_times_hr=df_hr.reindex(psst.index)
df_times_hr=df_times_hr.groupby('clinical_event_member_id').ffill().fillna(-10000-1).reset_index().set_index(['clinical_event_member_id','tdindex'])
df_times_hr=(-(df_times_hr.sub(df_times_hr.index.get_level_values(1),axis=0,level=1))).clip(upper=10000).astype(np.int)
df_times_hr=df_times_hr.replace(10000,-999)
df_times_hr

<ipython-input-4-1b42989eda2d>:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_times_hr=(-(df_times_hr.sub(df_times_hr.index.get_level_values(1),axis=0,level=1))).clip(upper=10000).astype(np.int)


ANNUAL VISIT_lv  ATRIAL FIBRILLATION_lv  \
clinical_event_member_id tdindex                                            
1                        0                   -999                    -999   
                         30                  -999                    -999   
                         60                    19                    -999   
                         90                    49                    -999   
                         120                   79                    -999   
...                                           ...                     ...   
200000                   450                  323                    -999   
                         480                  353                    -999   
                         510                  383                    -999   
                         540                  413                    -999   
                         570                  443                    -999   

                                  DIABETES MANAGEMENT VISIT_lv  \
clinical_event_member_id tdindex                                 
1                        0                                -999   
                         30                               -999   
                         60                               -999   
                         90                               -999   
                         120                              -999   
...                                                        ...   
200000                   450                              -999   
                         480                              -999   
                         510                              -999   
                         540                              -999   
                         570                              -999   

                                  HEART FAILURE HOSPITALIZATION_lv  \
clinical_event_member_id tdindex                                     
1                        0                                    -999   
                         30                                   -999   
                         60                                   -999   
                         90                                   -999   
                         120                                  -999   
...                                                            ...   
200000                   450                                  -999   
                         480                                  -999   
                         510                                  -999   
                         540                                  -999   
                         570                                  -999   

                                  HYPERTENSION MANAGEMENT VISIT_lv  \
clinical_event_member_id tdindex                                     
1                        0                                    -999   
                         30                                   -999   
                         60                                   -999   
                         90                                   -999   
                         120                                     4   
...                                                            ...   
200000                   450                                  -999   
                         480                                  -999   
                         510                                  -999   
                         540                                  -999   
                         570                                  -999   

                                  STROKE HOSPITALIZATION_lv  
clinical_event_member_id tdindex                             
1                        0                             -999  
                         30                            -999  
                         60                            -999  
                         90              

## number of counts of visit at each snapshot

In [5]:
df_hr=pd.concat(cat_frames,axis=1,ignore_index=True)


df_hr = df_hr.reset_index()
df_hr.clinical_event_time=np.ceil(df_hr.clinical_event_time/30.0)*30
df_hr=df_hr.set_index(['clinical_event_member_id','clinical_event_time'])
df_counts=df_hr.notnull().astype(np.int) 
df_counts=df_counts.groupby('clinical_event_member_id').cumsum().reset_index()

d = {x:"last" for x in df_counts.columns}
df_counts = df_counts.groupby(['clinical_event_member_id','clinical_event_time'], as_index=True).aggregate(d).reindex(columns=df_counts.columns)
del df_counts['clinical_event_member_id'],df_counts['clinical_event_time']
df_counts.columns=[x for y in cat_frames for x in y.columns.get_level_values(0)]

df_counts=df_counts.add_suffix('_cnt')
df_counts = df_counts.reindex(psst.index).ffill().fillna(0)

<ipython-input-5-42533913c4cc>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_counts=df_hr.notnull().astype(np.int)


## get counts of visit at each snapshot for every rolling 30,60,90,120,150,180 days

In [7]:
df_rolling_counts = pd.concat([df_counts.groupby(level=0).diff(x).fillna(df_counts).add_suffix('_'+str(x*30)+'_daysago') for x in [1,2,3,4,5,6]],axis=1)

In [8]:
df_rolling_counts

ANNUAL VISIT_cnt_30_daysago  \
clinical_event_member_id tdindex                                
1                        0                                0.0   
                         30                               0.0   
                         60                               1.0   
                         90                               0.0   
                         120                              0.0   
...                                                       ...   
200000                   450                              0.0   
                         480                              0.0   
                         510                              0.0   
                         540                              0.0   
                         570                              0.0   

                                  ATRIAL FIBRILLATION_cnt_30_daysago  \
clinical_event_member_id tdindex                                       
1                        0                                       0.0   
                         30                                      0.0   
                         60                                      0.0   
                         90                                      0.0   
                         120                                     0.0   
...                                                              ...   
200000                   450                                     0.0   
                         480                                     0.0   
                         510                                     0.0   
                         540                                     0.0   
                         570                                     0.0   

                                  DIABETES MANAGEMENT VISIT_cnt_30_daysago  \
clinical_event_member_id tdindex                                             
1                        0                                             0.0   
                         30                                            0.0   
                         60                                            0.0   
                         90                                            0.0   
                         120                                           0.0   
...                                                                    ...   
200000                   450                                           0.0   
                         480                                           0.0   
                         510                                           0.0   
                         540                                           0.0   
                         570                                           0.0   

                                  HEART FAILURE HOSPITALIZATION_cnt_30_daysago  \
clinical_event_member_id tdindex                                                 
1                        0                                                 0.0   
                         30                                                0.0   
                         60                                                0.0   
                         90                                                0.0   
                         120                                               0.0   
...                                                                        ...   
200000                   450                                               0.0   
                         480                                               0.0   
                         510                                               0.0   
                         540                                               0.0   
                         570                                               0.0   

                                  HYPERTENSION MANAGEMENT VISIT_cnt_30_daysago  \
clinical_event_member_id tdindex                                                 
1    

In [ ]:
df_rolling_counts = pd.concat([psst,df_times_hr,df_counts,df_rolling_counts],axis=1)

In [ ]:
df_rolling_counts.to_csv("feature_engineered_dataset.csv.gz")